In [7]:
# Install Faiss
!pip install faiss-gpu

# Install Pyserini
!pip install pyserini

In [8]:

# Data Prep
!wget https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip -P collections
!unzip collections/nfcorpus.zip -d collections


--2024-02-14 22:52:47--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2448432 (2.3M) [application/zip]
Saving to: ‘collections/nfcorpus.zip.2’

nfcorpus.zip.2      100%[===================>]   2.33M  2.26MB/s    in 1.0s    

2024-02-14 22:52:49 (2.26 MB/s) - ‘collections/nfcorpus.zip.2’ saved [2448432/2448432]

Archive:  collections/nfcorpus.zip
replace collections/nfcorpus/qrels/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: َA
error:  invalid response [َA]
replace collections/nfcorpus/qrels/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: collections/nfcorpus/qrels/train.tsv  
  inflating: collections/nfcorpus/qrels/test.tsv  
  inflating: collections

In [9]:

# Query data munging
import json

with open('collections/nfcorpus/queries.tsv', 'w') as out:
    with open('collections/nfcorpus/queries.jsonl', 'r') as f:
        for line in f:
            l = json.loads(line)
            out.write(l['_id'] + '\t' + l['text'] + '\n')

In [10]:



# Relevance judgments (qrels) data munging
!tail -n +2 collections/nfcorpus/qrels/test.tsv | sed 's/\t/\tQ0\t/' > collections/nfcorpus/qrels/test.qrels


In [11]:

# Indexing
!python -m pyserini.encode \
  input   --corpus collections/nfcorpus/corpus.jsonl \
          --fields title text \
  output  --embeddings indexes/faiss.nfcorpus.contriever-msmacro \
          --to-faiss \
  encoder --encoder facebook/contriever-msmarco \
          --device cpu \
          --pooling mean \
          --fields title text \
          --batch 32




3633it [00:00, 107927.89it/s]
100% 114/114 [28:18<00:00, 14.90s/it]


In [12]:
import faiss

index = faiss.read_index('indexes/faiss.nfcorpus.contriever-msmacro/index')
num_vectors = index.ntotal

In [13]:
for i in range(num_vectors):
    vector = index.reconstruct(i)
    print(f"Vector {i}: {vector}")

Streaming output truncated to the last 5000 lines.
  0.01618226 -0.00174899  0.00773179 -0.02491117  0.01178309  0.10834692
  0.02582213  0.05744912 -0.035428    0.04176953  0.04453957 -0.03635105
 -0.01534521 -0.02525122 -0.00604546  0.02696695  0.11226736  0.02081496
 -0.06020914 -0.03108971 -0.00181534 -0.07343625 -0.02888012  0.01816109
  0.05882426 -0.09578757  0.01984956  0.01010741 -0.00338439  0.02734621
 -0.12450624  0.0101505   0.44392222  0.0816242  -0.02498854  0.04964343
  0.01118695 -0.05513201 -0.01646433  0.00332079  0.04581673  0.03716886
 -0.00668258  0.01939672  0.02234914 -0.02493415 -0.05552603  0.05200051]
Vector 188: [-7.65931532e-02  2.62899566e-02 -1.42312264e-02 -1.19340293e-01
 -2.50096656e-02  1.97532978e-02  2.79805772e-02  3.23230773e-02
  2.23679151e-02 -1.37878594e-03 -8.31198841e-02 -6.30569533e-02
  1.26260798e-02  3.06414943e-02  2.90410090e-02 -8.50523710e-02
 -3.04328874e-02  1.78651556e-01  4.49828357e-02  4.52646893e-03
 -6.55600429e-02 -6.5099313

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
  3.88730653e-02 -2.20722836e-02  5.22115119e-02  5.70515543e-02
 -1.41660552e-02 -5.39935082e-02  1.15353055e-01 -1.01081328e-03
  5.50047308e-03  9.56473574e-02  3.81118581e-02 -6.54865056e-02
 -6.45309910e-02 -4.25275154e-02  2.43905559e-02 -6.98829591e-02
  4.38185424e-01 -1.01533644e-01 -4.05643843e-02 -1.99114867e-02
 -6.20531924e-02  7.01870816e-03  3.05870976e-02 -4.08118367e-02
  1.67332888e-02  5.74883334e-02 -2.08888613e-02  4.28796075e-02
  4.76109833e-02 -5.76208830e-02  1.57639626e-02  7.70066530e-02]
Vector 834: [ 3.04041170e-02 -6.34244382e-02 -5.98411672e-02  4.91063483e-02
 -1.56627715e-01  2.35326849e-02 -3.39404084e-02 -2.01887675e-02
  2.56164838e-02 -3.01252715e-02 -2.84672398e-02 -7.06366971e-02
 -8.34534019e-02 -8.95894915e-02  3.86395231e-02 -4.42202352e-02
 -4.00469676e-02  5.03796227e-02  1.61779597e-02  2.78505608e-02
 -1.89201739e-02  6.21587858e-02  1.35490317e-02  5.57013154e-02
  6.02865778e-02 -8.515749

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
 -1.92978587e-02  2.26263353e-03  3.71916704e-02 -1.12339919e-02
 -3.00922489e-04 -2.72064209e-02  6.74036471e-03  8.02003592e-03
  4.50648777e-02  3.95437256e-02 -1.37674175e-02  4.91179861e-02
 -9.19190049e-02 -8.74458775e-02  1.67133398e-02  4.83455248e-02
  4.85025913e-01  5.34334313e-03 -6.87922770e-03  5.64488731e-02
 -4.88880984e-02 -9.03695449e-02 -5.15068062e-02  1.71276648e-02
  1.07931472e-01  3.38889994e-02 -5.29109836e-02  3.44649777e-02
 -9.10681263e-02 -3.35366502e-02 -7.61283264e-02  1.53197885e-01]
Vector 1373: [-1.05905030e-02 -5.55188209e-02  2.31231675e-02 -7.10272565e-02
 -3.26671936e-02  3.22523415e-02  4.65843678e-02 -7.40739703e-02
 -4.75092884e-03  5.12475148e-02 -9.57849026e-02  4.58541326e-02
 -8.57589487e-03  1.45596918e-02 -2.38285065e-02  3.11633665e-02
 -4.85407794e-03  9.77426842e-02 -2.24697702e-02 -1.42688686e-02
 -4.38175537e-03  2.45080590e-02 -7.07629472e-02 -5.59972376e-02
 -8.25761706e-02 -4.06335

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
  3.94171327e-02 -9.39827189e-02  4.97516617e-02 -1.42199988e-03
  9.90806296e-02 -4.15919721e-02  5.56690358e-02 -6.18053228e-03
 -1.65839344e-02 -8.44529048e-02  4.99637537e-02 -1.04033768e-01
 -8.20614845e-02 -5.57278469e-03 -8.82738903e-02  1.11413328e-02
  4.58348334e-01 -7.97353033e-03  1.84729397e-02 -2.08456758e-02
 -6.22851867e-03  3.98313701e-02  2.04756986e-02  8.40109885e-02
 -4.85113673e-02  2.00327709e-02 -5.44763077e-03  7.87436292e-02
  6.87388182e-02 -2.33205929e-02  7.64761195e-02 -2.59539746e-02]
Vector 3077: [-9.80244055e-02 -6.80063516e-02 -4.90586087e-02  1.23072334e-01
 -1.04419403e-01  1.17385178e-03 -6.95303231e-02 -7.68806264e-02
  5.24384491e-02  2.01531891e-02 -8.10373202e-02  6.15910403e-02
  4.42196429e-02  1.98036730e-02 -1.44574908e-04 -4.72072698e-02
  3.00479420e-02  1.10338435e-01 -5.01271114e-02  1.61025394e-02
  6.88094720e-02  8.40122700e-02 -8.57080370e-02  1.79913044e-02
  2.64039841e-02 -1.59226

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
  1.08307321e-02  7.10706739e-03  5.57580963e-03 -7.79759744e-03
  1.80322826e-02 -7.14332461e-02 -6.34303987e-02  2.19156258e-02
 -2.53528524e-02 -1.36133246e-02 -5.95506467e-02 -4.07023355e-02
 -3.16527002e-02 -2.85129678e-02  3.36852074e-02 -2.65442748e-02
  4.38319176e-01  7.54141882e-02  8.13265443e-02  4.38393392e-02
  6.24624006e-02  2.10911725e-02 -1.20912857e-01 -7.95187950e-02
  2.18765773e-02  4.07521091e-02 -4.38944250e-02 -2.47060340e-02
 -2.66825990e-03  3.09446305e-02 -2.14161277e-02  3.25112343e-02]
Vector 3533: [-2.25860905e-02 -4.75130267e-02 -6.50529861e-02 -2.08826968e-03
 -4.28610072e-02 -4.83189300e-02 -6.98918402e-02  2.25202702e-02
  1.41996583e-02  7.01577365e-02 -3.92831154e-02 -4.36512530e-02
 -4.43387702e-02 -3.58797796e-02  1.43787940e-03  1.35105342e-01
 -2.29855888e-02  8.29940438e-02 -1.99732874e-02 -2.31529400e-02
  2.18444820e-02 -7.45976791e-02 -3.15900743e-02  1.70545261e-02
  4.42404896e-02 -3.38433

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
docids = []
with open('indexes/faiss.nfcorpus.contriever-msmacro/docid', 'r') as fin:
    docids = [line.rstrip() for line in fin.readlines()]

v1 = index.reconstruct(docids.index('MED-4555'))

In [15]:
# This is the string contents of doc MED-4555
doc_text = 'Analysis of risk factors for abdominal aortic aneurysm in a cohort of more than 3 million individuals. BACKGROUND: Abdominal aortic aneurysm (AAA) disease is an insidious condition with an 85% chance of death after rupture. Ultrasound screening can reduce mortality, but its use is advocated only for a limited subset of the population at risk. METHODS: We used data from a retrospective cohort of 3.1 million patients who completed a medical and lifestyle questionnaire and were evaluated by ultrasound imaging for the presence of AAA by Life Line Screening in 2003 to 2008. Risk factors associated with AAA were identified using multivariable logistic regression analysis. RESULTS: We observed a positive association with increasing years of smoking and cigarettes smoked and a negative association with smoking cessation. Excess weight was associated with increased risk, whereas exercise and consumption of nuts, vegetables, and fruits were associated with reduced risk. Blacks, Hispanics, and Asians had lower risk of AAA than whites and Native Americans. Well-known risk factors were reaffirmed, including male gender, age, family history, and cardiovascular disease. A predictive scoring system was created that identifies aneurysms more efficiently than current criteria and includes women, nonsmokers, and individuals aged <65 years. Using this model on national statistics of risk factors prevalence, we estimated 1.1 million AAAs in the United States, of which 569,000 are among women, nonsmokers, and individuals aged <65 years. CONCLUSIONS: Smoking cessation and a healthy lifestyle are associated with lower risk of AAA. We estimated that about half of the patients with AAA disease are not eligible for screening under current guidelines. We have created a high-yield screening algorithm that expands the target population for screening by including at-risk individuals not identified with existing screening criteria.'

from pyserini.encode import AutoDocumentEncoder
encoder = AutoDocumentEncoder('facebook/contriever-msmarco', device='cpu', pooling='mean')

v2 = encoder.encode(doc_text)

In [16]:
import numpy as np
np.linalg.norm(v2[0] - v1)

0.0

In [17]:
# Install libomp-dev and faiss-cpu
!apt-get install libomp-dev
!pip install faiss-cpu




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [18]:
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder

encoder = AutoQueryEncoder('facebook/contriever-msmarco', device='cpu', pooling='mean')
searcher = FaissSearcher('indexes/faiss.nfcorpus.contriever-msmacro', encoder)
hits = searcher.search('How to Help Prevent Abdominal Aortic Aneurysms')

for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.6f}')

 1 MED-4555 1.472202
 2 MED-3180 1.125015
 3 MED-1309 1.067154
 4 MED-2224 1.059537
 5 MED-4423 1.038440
 6 MED-4887 1.032622
 7 MED-2530 1.020758
 8 MED-2372 1.016142
 9 MED-1006 1.013599
10 MED-2587 1.010812


In [19]:
from pyserini.encode import AutoQueryEncoder

q_encoder = AutoQueryEncoder('facebook/contriever-msmarco', device='cpu', pooling='mean')
q_vec = q_encoder.encode('How to Help Prevent Abdominal Aortic Aneurysms')

In [20]:
import numpy as np

np.dot(q_vec, v1)

1.4722017

In [21]:
from tqdm import tqdm

scores = []
# Iterate through all document vectors and compute dot product.
for i in tqdm(range(num_vectors)):
    vector = index.reconstruct(i)
    score = np.dot(q_vec, vector)
    scores.append([docids[i], score])

# Sort by score descending.
scores.sort(key=lambda x: -x[1])

for s in scores[:10]:
    print(f'{s[0]} {s[1]:.6f}')

100%|██████████| 3633/3633 [00:00<00:00, 18801.76it/s]

MED-4555 1.472202
MED-3180 1.125015
MED-1309 1.067154
MED-2224 1.059537
MED-4423 1.038440
MED-4887 1.032622
MED-2530 1.020758
MED-2372 1.016142
MED-1006 1.013599
MED-2587 1.010811


In [22]:
!mkdir collections/nfcorpus/pyserini-corpus

In [23]:
import json

with open('collections/nfcorpus/pyserini-corpus/corpus.jsonl', 'w') as out:
    with open('collections/nfcorpus/corpus.jsonl', 'r') as f:
        for line in f:
            l = json.loads(line)
            s = json.dumps({'id': l['_id'], 'contents': l['title'] + ' ' + l['text']})
            out.write(s + '\n')

In [25]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/nfcorpus/pyserini-corpus/ \
  --index indexes/lucene.nfcorpus \
  --generator DefaultLuceneDocumentGenerator \
  --storePositions --storeDocvectors --storeRaw

2024-02-14 23:28:17,900 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-02-14 23:28:17,908 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-02-14 23:28:17,908 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-02-14 23:28:17,908 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: collections/nfcorpus/pyserini-corpus/
2024-02-14 23:28:17,908 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-02-14 23:28:17,909 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: indexes/lucene.nfcorpus
2024-02-14 23:28:17,909 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 4
2024-02-14 23:28:17,910 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false


In [26]:
!python -m pyserini.search.lucene \
  --index indexes/lucene.nfcorpus \
  --topics collections/nfcorpus/queries.tsv \
  --output runs/run.beir-bm25.nfcorpus.txt \
  --hits 1000 --bm25 \
  --threads 4 --batch-size 16

Running collections/nfcorpus/queries.tsv topics, saving to runs/run.beir-bm25.nfcorpus.txt...
100% 3237/3237 [01:49<00:00, 29.67it/s]


In [27]:
!python -m pyserini.eval.trec_eval \
  -c -m ndcg_cut.10 collections/nfcorpus/qrels/test.qrels \
  runs/run.beir-bm25.nfcorpus.txt

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 4.08MB/s]                
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/nfcorpus/qrels/test.qrels', 'runs/run.beir-bm25.nfcorpus.txt']
Results:
ndcg_cut_10           	all	0.3218


In [28]:
from pyserini.index.lucene import IndexReader
import json

index_reader = IndexReader('indexes/lucene.nfcorpus')
tf = index_reader.get_document_vector('MED-4555')
bm25_weights = \
    {term: index_reader.compute_bm25_term_weight('MED-4555', term, analyzer=None) \
     for term in tf.keys()}

print(json.dumps(bm25_weights, indent=4, sort_keys=True))

{
    "1.1": 2.2132339477539062,
    "2003": 2.0071027278900146,
    "2008": 1.9285697937011719,
    "3": 0.8006289005279541,
    "3.1": 2.387873649597168,
    "569,000": 3.995551586151123,
    "65": 2.5261287689208984,
    "85": 2.2027695178985596,
    "aaa": 6.861149311065674,
    "abdomin": 2.810744047164917,
    "about": 1.2124857902526855,
    "advoc": 2.9084181785583496,
    "after": 0.6753702163696289,
    "ag": 0.9848723411560059,
    "algorithm": 2.9084181785583496,
    "american": 1.5901331901550293,
    "among": 0.8441131114959717,
    "analysi": 1.1517744064331055,
    "aneurysm": 5.2744975090026855,
    "aortic": 3.8452444076538086,
    "asian": 2.3056604862213135,
    "associ": 0.7863264083862305,
    "background": 0.8268404006958008,
    "black": 1.860482931137085,
    "can": 0.8828715085983276,
    "cardiovascular": 1.1131560802459717,
    "cessat": 3.4776077270507812,
    "chanc": 2.563375949859619,
    "cigarett": 2.2459728717803955,
    "cohort": 1.5951440334320068,


In [29]:
from pyserini.analysis import Analyzer, get_lucene_analyzer

analyzer = Analyzer(get_lucene_analyzer())
query_tokens = analyzer.analyze('How to Help Prevent Abdominal Aortic Aneurysms')
multihot_query_weights = {k: 1 for k in query_tokens}

In [30]:
sum({term: bm25_weights[term] \
     for term in bm25_weights.keys() & \
     multihot_query_weights.keys()}.values())

11.930485963821411

In [31]:
def dot(q_weights, d_weights):
    return sum({term: d_weights[term] \
                for term in d_weights.keys() & \
                q_weights.keys()}.values())

dot(multihot_query_weights, bm25_weights)

11.930485963821411

In [32]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
from tqdm import tqdm

searcher = LuceneSearcher('indexes/lucene.nfcorpus')
index_reader = IndexReader('indexes/lucene.nfcorpus')

scores = []
# Iterate through all docids in the index.
for i in tqdm(range(0, searcher.num_docs)):
    docid = searcher.doc(i).get('id')
    # Reconstruct the BM25 document vector.
    tf = index_reader.get_document_vector(docid)
    bm25_weights = \
        {term: index_reader.compute_bm25_term_weight(docid, term, analyzer=None) \
         for term in tf.keys()}
    # Compute and retain the query-document score.
    score = dot(multihot_query_weights, bm25_weights)
    scores.append([docid, score])

# Sort by score descending.
scores.sort(key=lambda x: -x[1])

for s in scores[:10]:
    print(f'{s[0]} {s[1]:.4f}')

100%|██████████| 3633/3633 [00:20<00:00, 178.83it/s]

MED-4555 11.9305
MED-4423 8.4771
MED-3180 7.1896
MED-2718 6.0102
MED-1309 5.8181
MED-4424 5.7448
MED-1705 5.6101
MED-4902 5.3639
MED-1009 5.2533
MED-1512 5.2068
